## Решение практической задачи 2

### Задача: прогнозирование потребительских предпочтений на промышленные товары

<center>
    <img src="https://raw.githubusercontent.com/aimclub/OCEANAI/main/docs/source/user_guide/notebooks/images/Step_1_get_scores_ru.svg" width="60%"/>
</center>

<center>
    <img src="https://raw.githubusercontent.com/aimclub/OCEANAI/main/docs/source/user_guide/notebooks/images/Step_2_task_2_ru.svg" width="60%"/>
</center>

<hr>

In [1]:
import os  # Взаимодействие с файловой системой
import sys # Доступ к некоторым переменным и функциям Python

PATH_TO_SOURCE = os.path.abspath(os.path.dirname(globals()['_dh'][0]))
PATH_TO_ROOT = os.path.join(PATH_TO_SOURCE, '..', '..', '..')

sys.path.insert(0, os.path.abspath(PATH_TO_ROOT))

### `FI V2`

In [2]:
# Импорт необходимых инструментов
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

corpus = 'fi'

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang='en')
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url)

url = _b5.weights_for_big5_['video'][corpus]['fe']['sberdisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url)

url = _b5.weights_for_big5_['video'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
res_setup_translation_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['sberdisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url)

url = _b5.weights_for_big5_['text'][corpus]['nn']['sberdisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['sberdisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url)

PATH_TO_DIR = './video_FI/'
PATH_SAVE_VIDEO = './video_FI/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '429713680?token=FqHdMLSSh7zYSZt&filename=_plk5k7PBEg.003.mp4',
    '429713681?token=Hz9b4lQkrLfic33&filename=be0DQawtVkE.002.mp4',
    '429713683?token=EgUXS9Xs8xHm5gz&filename=2d6btbaNdfo.000.mp4',
    '429713684?token=1U26753kmPYdIgt&filename=300gK3CnzW0.003.mp4',
    '429713685?token=LyigAWLTzDNwKJO&filename=300gK3CnzW0.001.mp4',
    '429713686?token=EpfRbCKHyuc4HPu&filename=cLaZxEf1nE4.004.mp4',
    '429713687?token=FNTkwqBr4jOS95l&filename=g24JGYuT74A.004.mp4',
    '429713688?token=qDT95nz7hfm2Nki&filename=JZNMxa3OKHY.000.mp4',
    '429713689?token=noLguEGXDpbcKhg&filename=nvlqJbHk_Lc.003.mp4',
    '429713679?token=9L7RQ0hgdJlcek6&filename=4vdJGgZpj4k.003.mp4'
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mp4'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_[corpus]['sberdisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = 'en')

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:49:37</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:49:40</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_FI\test\_plk5k7PBEg.003.mp4 ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.581159,0.628822,0.466609,0.622129,0.553832
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.463991,0.418851,0.41301,0.493329,0.423093
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.454281,0.415049,0.39189,0.485114,0.420741
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.588461,0.643233,0.530789,0.603038,0.593398
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.633433,0.533295,0.523742,0.608591,0.588456
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.636944,0.542386,0.558461,0.570975,0.558983
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.531518,0.376987,0.393309,0.4904,0.447881
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.610342,0.541418,0.563163,0.595013,0.569461
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.495809,0.458526,0.414436,0.469152,0.435461


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:49:40</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.0589,0.0612,0.0864,0.0697,0.0582,0.0669
Accuracy,0.9411,0.9388,0.9136,0.9303,0.9418,0.9331


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:49:40</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0669</span>, средняя точность: <span style="color:#1776D2">0.9331</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 63.043 сек. ---**</span>

True

<hr>

Для прогнозирования потребительских предпочтений в промышленных товарах необходимо знать коэффициенты корреляции, определяющие взаимосвязь между персональными качествами личности человека и предпочтениями в товарах или услугах.

В качестве примера предлагается использование коэффициентов корреляции между персональными качествами человека и характеристиками автомобилей, представленными в статье:1) 

O'Connor P. J. et al. What Drives Consumer Automobile Choice? Investigating Personality Trait Predictors of Vehicle Preference Factors // Personality and Individual Differences. – 2022. – Vol. 184. – pp. 111
220.
Пользователь может установить свои коэффициенты корреляции.

#### Прогнозирование потребительских предпочтений на характеристики атомобиля

In [3]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478675818?token=EjfLMqOeK8cfnOu&filename=auto_characteristics.csv'
df_correlation_coefficients = pd.read_csv(url)
df_correlation_coefficients = pd.DataFrame(
    df_correlation_coefficients.drop(['Style and performance', 'Safety and practicality'], axis = 1)
)
df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,Performance,Classic car features,Luxury additions,Fashion and attention,Recreation,Technology,Family friendly,Safe and reliable,Practical and easy to use,Economical/low cost,Basic features
ID,,,,,,,,,,,,
1,Openness,0.020000,-0.033333,-0.030000,-0.050000,0.033333,0.013333,-0.030000,0.136667,0.106667,0.093333,0.006667
2,Conscientiousness,0.013333,-0.193333,-0.063333,-0.096667,-0.096667,0.086667,-0.063333,0.280000,0.180000,0.130000,0.143333
3,Extraversion,0.133333,0.060000,0.106667,0.123333,0.126667,0.120000,0.090000,0.136667,0.043333,0.073333,0.050000
4,Agreeableness,-0.036667,-0.193333,-0.133333,-0.133333,-0.090000,0.046667,-0.016667,0.240000,0.160000,0.120000,0.083333
5,Non-Neuroticism,0.016667,-0.006667,-0.010000,-0.006667,-0.033333,0.046667,-0.023333,0.093333,0.046667,0.046667,-0.040000


In [4]:
_b5._priority_calculation(
    correlation_coefficients = df_correlation_coefficients,
    col_name_ocean = 'Trait',
    threshold = 0.55,
    number_priority = 3,
    number_importance_traits = 3,
    out = False
)

_b5._save_logs(df = _b5.df_files_priority_, name = 'auto_characteristics_priorities_fi_en', out = True)

_b5.df_files_priority_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Priority 1,Priority 2,Priority 3,Trait importance 1,Trait importance 2,Trait importance 3
ID,,,,,,,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.581159,0.628822,0.466609,0.622129,0.553832,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.463991,0.418851,0.41301,0.493329,0.423093,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.454281,0.415049,0.39189,0.485114,0.420741,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.588461,0.643233,0.530789,0.603038,0.593398,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.633433,0.533295,0.523742,0.608591,0.588456,Practical and easy to use,Safe and reliable,Economical/low cost,Agreeableness,Openness,Non-Neuroticism
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.636944,0.542386,0.558461,0.570975,0.558983,Safe and reliable,Economical/low cost,Practical and easy to use,Agreeableness,Openness,Extraversion
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.531518,0.376987,0.393309,0.4904,0.447881,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.610342,0.541418,0.563163,0.595013,0.569461,Safe and reliable,Economical/low cost,Practical and easy to use,Agreeableness,Openness,Extraversion
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.495809,0.458526,0.414436,0.469152,0.435461,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness


#### Прогнозирование потребительских предпочтений на характеристики мобильного девайса

В качестве примера предлагается использование коэффициентов корреляции между персональными качествами человека и характеристиками автомобилей, представленными в статье:

1) Peltonen E., Sharmila P., Asare K. O., Visuri A., Lagerspetz E., Ferreira D. (2020). When phones get personal: Predicting Big Five personality traits from application usage // Pervasive and Mobile Computing. – 2020. – Vol. 69. – 101269.

In [5]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478676690?token=7KcAxPqMpWiYQnx&filename=divice_characteristics.csv'
df_divice_characteristics = pd.read_csv(url)

df_divice_characteristics.index.name = 'ID'
df_divice_characteristics.index += 1
df_divice_characteristics.index = df_divice_characteristics.index.map(str)

df_divice_characteristics

,Trait,Communication,Game Action,Game Board,Game Casino,Game Educational,Game Simulation,Game Trivia,Entertainment,Finance,Health and Fitness,Media and Video,Music and Audio,News and Magazines,Personalisation,Travel and Local,Weather
ID,,,,,,,,,,,,,,,,,
1,Openness,0.118,0.056,0.079,0.342,0.027,0.104,0.026,0.000,0.006,0.002,0.000,0.000,0.001,0.004,0.002,0.004
2,Conscientiousness,0.119,0.043,0.107,0.448,0.039,0.012,0.119,0.000,0.005,0.001,0.000,0.002,0.002,0.001,0.001,0.003
3,Extraversion,0.246,0.182,0.211,0.311,0.102,0.165,0.223,0.001,0.003,0.000,0.001,0.001,0.001,0.004,0.009,0.003
4,Agreeableness,0.218,0.104,0.164,0.284,0.165,0.122,0.162,0.000,0.003,0.001,0.000,0.002,0.002,0.001,0.004,0.003
5,Non-Neuroticism,0.046,0.047,0.125,0.515,0.272,0.179,0.214,0.002,0.030,0.001,0.000,0.005,0.003,0.008,0.004,0.007


In [6]:
_b5._priority_calculation(
    correlation_coefficients = df_divice_characteristics,
    col_name_ocean = 'Trait',
    threshold = 0.55,
    number_priority = 3,
    number_importance_traits = 3,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_, name = 'divice_characteristics_priorities_fi_en', out = True)

_b5.df_files_priority_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Priority 1,Priority 2,Priority 3,Trait importance 1,Trait importance 2,Trait importance 3
ID,,,,,,,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.581159,0.628822,0.466609,0.622129,0.553832,Game Casino,Game Educational,Game Trivia,Non-Neuroticism,Conscientiousness,Agreeableness
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.463991,0.418851,0.41301,0.493329,0.423093,Media and Video,Entertainment,Health and Fitness,Conscientiousness,Agreeableness,Extraversion
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.454281,0.415049,0.39189,0.485114,0.420741,Media and Video,Entertainment,Health and Fitness,Conscientiousness,Agreeableness,Extraversion
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.588461,0.643233,0.530789,0.603038,0.593398,Game Casino,Game Educational,Game Trivia,Non-Neuroticism,Conscientiousness,Agreeableness
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.633433,0.533295,0.523742,0.608591,0.588456,Game Casino,Game Educational,Game Simulation,Non-Neuroticism,Agreeableness,Openness
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.636944,0.542386,0.558461,0.570975,0.558983,Game Casino,Game Simulation,Game Educational,Non-Neuroticism,Agreeableness,Extraversion
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.531518,0.376987,0.393309,0.4904,0.447881,Media and Video,Entertainment,Health and Fitness,Conscientiousness,Agreeableness,Extraversion
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.610342,0.541418,0.563163,0.595013,0.569461,Game Casino,Game Simulation,Game Educational,Non-Neuroticism,Agreeableness,Extraversion
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.495809,0.458526,0.414436,0.469152,0.435461,Media and Video,Entertainment,Health and Fitness,Conscientiousness,Agreeableness,Extraversion


### `MuPTA` (ru)

In [7]:
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

corpus = 'mupta'
lang = 'ru'

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang=lang)
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url)

url = _b5.weights_for_big5_['video'][corpus]['fe']['sberdisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url)

url = _b5.weights_for_big5_['video'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
res_setup_translation_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['sberdisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url)

url = _b5.weights_for_big5_['text'][corpus]['nn']['sberdisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['sberdisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url)

PATH_TO_DIR = './video_MuPTA/'
PATH_SAVE_VIDEO = './video_MuPTA/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '477995979?token=2cvyk7CS0mHx2MJ&filename=speaker_06_center_83.mov',
    '477995980?token=jGPtBPS69uzFU6Y&filename=speaker_01_center_83.mov',
    '477995967?token=zCaRbNB6ht5wMPq&filename=speaker_11_center_83.mov',
    '477995966?token=B1rbinDYRQKrI3T&filename=speaker_15_center_83.mov',
    '477995978?token=dEpVDtZg1EQiEQ9&filename=speaker_07_center_83.mov',
    '477995961?token=o1hVjw8G45q9L9Z&filename=speaker_19_center_83.mov',
    '477995964?token=5K220Aqf673VHPq&filename=speaker_23_center_83.mov',
    '477995965?token=v1LVD2KT1cU7Lpb&filename=speaker_24_center_83.mov',
    '477995962?token=tmaSGyyWLA6XCy9&filename=speaker_27_center_83.mov',
    '477995963?token=bTpo96qNDPcwGqb&filename=speaker_10_center_83.mov',
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mov'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['mupta']['sberdisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = lang)

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:57:36</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:57:40</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_MuPTA\test\speaker_27_center_83.mov ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.758137,0.693356,0.650108,0.744589,0.488671
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.681602,0.654339,0.607156,0.731282,0.417908
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.666104,0.656836,0.567863,0.685067,0.378102
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.694171,0.596195,0.571414,0.66223,0.348639
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.712885,0.594764,0.571709,0.716696,0.37802
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.664158,0.670411,0.60421,0.696056,0.399842
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.761213,0.652635,0.651028,0.788677,0.459676
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.692788,0.68324,0.616737,0.795205,0.447242
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.705923,0.658382,0.610645,0.697415,0.411988


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:57:40</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.0673,0.0789,0.1325,0.102,0.1002,0.0962
Accuracy,0.9327,0.9211,0.8675,0.898,0.8998,0.9038


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 19:57:40</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0962</span>, средняя точность: <span style="color:#1776D2">0.9038</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 408.51 сек. ---**</span>

True

<hr>

Для прогнозирования потребительских предпочтений в промышленных товарах необходимо знать коэффициенты корреляции, определяющие взаимосвязь между персональными качествами личности человека и предпочтениями в товарах или услугах.

В качестве примера предлагается использование коэффициентов корреляции между персональными качествами человека и характеристиками автомобилей, представленными в статье:

1) O'Connor P. J. et al. What Drives Consumer Automobile Choice? Investigating Personality Trait Predictors of Vehicle Preference Factors // Personality and Individual Differences. – 2022. – Vol. 184. – pp. 111220.

Пользователь может установить свои коэффициенты корреляции.

#### Прогнозирование потребительских предпочтений на характеристики атомобиля

In [8]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478675818?token=EjfLMqOeK8cfnOu&filename=auto_characteristics.csv'
df_correlation_coefficients = pd.read_csv(url)
df_correlation_coefficients = pd.DataFrame(
    df_correlation_coefficients.drop(['Style and performance', 'Safety and practicality'], axis = 1)
)
df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,Performance,Classic car features,Luxury additions,Fashion and attention,Recreation,Technology,Family friendly,Safe and reliable,Practical and easy to use,Economical/low cost,Basic features
ID,,,,,,,,,,,,
1,Openness,0.020000,-0.033333,-0.030000,-0.050000,0.033333,0.013333,-0.030000,0.136667,0.106667,0.093333,0.006667
2,Conscientiousness,0.013333,-0.193333,-0.063333,-0.096667,-0.096667,0.086667,-0.063333,0.280000,0.180000,0.130000,0.143333
3,Extraversion,0.133333,0.060000,0.106667,0.123333,0.126667,0.120000,0.090000,0.136667,0.043333,0.073333,0.050000
4,Agreeableness,-0.036667,-0.193333,-0.133333,-0.133333,-0.090000,0.046667,-0.016667,0.240000,0.160000,0.120000,0.083333
5,Non-Neuroticism,0.016667,-0.006667,-0.010000,-0.006667,-0.033333,0.046667,-0.023333,0.093333,0.046667,0.046667,-0.040000


In [9]:
_b5._priority_calculation(
    correlation_coefficients = df_correlation_coefficients,
    col_name_ocean = 'Trait',
    threshold = 0.55,
    number_priority = 3,
    number_importance_traits = 3,
    out = False
)

_b5._save_logs(df = _b5.df_files_priority_, name = 'auto_characteristics_priorities_mupta_ru', out = True)

_b5.df_files_priority_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Priority 1,Priority 2,Priority 3,Trait importance 1,Trait importance 2,Trait importance 3
ID,,,,,,,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.758137,0.693356,0.650108,0.744589,0.488671,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.681602,0.654339,0.607156,0.731282,0.417908,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.666104,0.656836,0.567863,0.685067,0.378102,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.694171,0.596195,0.571414,0.66223,0.348639,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.712885,0.594764,0.571709,0.716696,0.37802,Safe and reliable,Practical and easy to use,Economical/low cost,Agreeableness,Conscientiousness,Openness
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.664158,0.670411,0.60421,0.696056,0.399842,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.761213,0.652635,0.651028,0.788677,0.459676,Safe and reliable,Practical and easy to use,Economical/low cost,Agreeableness,Conscientiousness,Openness
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.692788,0.68324,0.616737,0.795205,0.447242,Safe and reliable,Practical and easy to use,Economical/low cost,Agreeableness,Conscientiousness,Openness
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.705923,0.658382,0.610645,0.697415,0.411988,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness


#### Прогнозирование потребительских предпочтений на характеристики мобильного девайса

В качестве примера предлагается использование коэффициентов корреляции между персональными качествами человека и характеристиками автомобилей, представленными в статье:

1) Peltonen E., Sharmila P., Asare K. O., Visuri A., Lagerspetz E., Ferreira D. (2020). When phones get personal: Predicting Big Five personality traits from application usage // Pervasive and Mobile Computing. – 2020. – Vol. 69. – 101269.

In [10]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478676690?token=7KcAxPqMpWiYQnx&filename=divice_characteristics.csv'
df_divice_characteristics = pd.read_csv(url)

df_divice_characteristics.index.name = 'ID'
df_divice_characteristics.index += 1
df_divice_characteristics.index = df_divice_characteristics.index.map(str)

df_divice_characteristics

,Trait,Communication,Game Action,Game Board,Game Casino,Game Educational,Game Simulation,Game Trivia,Entertainment,Finance,Health and Fitness,Media and Video,Music and Audio,News and Magazines,Personalisation,Travel and Local,Weather
ID,,,,,,,,,,,,,,,,,
1,Openness,0.118,0.056,0.079,0.342,0.027,0.104,0.026,0.000,0.006,0.002,0.000,0.000,0.001,0.004,0.002,0.004
2,Conscientiousness,0.119,0.043,0.107,0.448,0.039,0.012,0.119,0.000,0.005,0.001,0.000,0.002,0.002,0.001,0.001,0.003
3,Extraversion,0.246,0.182,0.211,0.311,0.102,0.165,0.223,0.001,0.003,0.000,0.001,0.001,0.001,0.004,0.009,0.003
4,Agreeableness,0.218,0.104,0.164,0.284,0.165,0.122,0.162,0.000,0.003,0.001,0.000,0.002,0.002,0.001,0.004,0.003
5,Non-Neuroticism,0.046,0.047,0.125,0.515,0.272,0.179,0.214,0.002,0.030,0.001,0.000,0.005,0.003,0.008,0.004,0.007


In [11]:
_b5._priority_calculation(
    correlation_coefficients = df_divice_characteristics,
    col_name_ocean = 'Trait',
    threshold = 0.55,
    number_priority = 3,
    number_importance_traits = 3,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_, name = 'divice_characteristics_priorities_mupta_ru', out = True)

_b5.df_files_priority_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Priority 1,Priority 2,Priority 3,Trait importance 1,Trait importance 2,Trait importance 3
ID,,,,,,,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.758137,0.693356,0.650108,0.744589,0.488671,Game Casino,Communication,Game Board,Extraversion,Agreeableness,Conscientiousness
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.681602,0.654339,0.607156,0.731282,0.417908,Game Casino,Communication,Game Board,Agreeableness,Extraversion,Conscientiousness
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.666104,0.656836,0.567863,0.685067,0.378102,Game Casino,Communication,Game Board,Agreeableness,Conscientiousness,Extraversion
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.694171,0.596195,0.571414,0.66223,0.348639,Game Casino,Communication,Game Board,Agreeableness,Extraversion,Conscientiousness
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.712885,0.594764,0.571709,0.716696,0.37802,Game Casino,Communication,Game Board,Agreeableness,Extraversion,Conscientiousness
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.664158,0.670411,0.60421,0.696056,0.399842,Game Casino,Communication,Game Board,Extraversion,Agreeableness,Conscientiousness
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.761213,0.652635,0.651028,0.788677,0.459676,Game Casino,Communication,Game Board,Agreeableness,Extraversion,Conscientiousness
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.692788,0.68324,0.616737,0.795205,0.447242,Game Casino,Communication,Game Board,Agreeableness,Extraversion,Conscientiousness
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.705923,0.658382,0.610645,0.697415,0.411988,Game Casino,Communication,Game Board,Extraversion,Agreeableness,Conscientiousness


### `MuPTA` (en)

In [12]:
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

corpus = 'fi'
lang = 'en'

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang=lang)
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['sberdisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url)

url = _b5.weights_for_big5_['video'][corpus]['fe']['sberdisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url)

url = _b5.weights_for_big5_['video'][corpus]['nn']['sberdisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
res_setup_translation_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['sberdisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url)

url = _b5.weights_for_big5_['text'][corpus]['nn']['sberdisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['sberdisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url)

PATH_TO_DIR = './video_MuPTA/'
PATH_SAVE_VIDEO = './video_MuPTA/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '477995979?token=2cvyk7CS0mHx2MJ&filename=speaker_06_center_83.mov',
    '477995980?token=jGPtBPS69uzFU6Y&filename=speaker_01_center_83.mov',
    '477995967?token=zCaRbNB6ht5wMPq&filename=speaker_11_center_83.mov',
    '477995966?token=B1rbinDYRQKrI3T&filename=speaker_15_center_83.mov',
    '477995978?token=dEpVDtZg1EQiEQ9&filename=speaker_07_center_83.mov',
    '477995961?token=o1hVjw8G45q9L9Z&filename=speaker_19_center_83.mov',
    '477995964?token=5K220Aqf673VHPq&filename=speaker_23_center_83.mov',
    '477995965?token=v1LVD2KT1cU7Lpb&filename=speaker_24_center_83.mov',
    '477995962?token=tmaSGyyWLA6XCy9&filename=speaker_27_center_83.mov',
    '477995963?token=bTpo96qNDPcwGqb&filename=speaker_10_center_83.mov',
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mov'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['mupta']['sberdisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = lang)

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 20:05:33</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 20:05:36</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_MuPTA\test\speaker_27_center_83.mov ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.564985,0.539052,0.440615,0.59251,0.488763
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.650774,0.663849,0.607308,0.643847,0.620627
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.435976,0.486683,0.313828,0.415446,0.396618
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.498542,0.511243,0.412592,0.468947,0.44399
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.394776,0.341608,0.327082,0.427304,0.354936
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.566107,0.543811,0.492766,0.587411,0.499433
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.506271,0.438215,0.430894,0.456177,0.44075
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.486463,0.521755,0.309894,0.432291,0.433601
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.417404,0.473339,0.320714,0.445086,0.414649


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 20:05:36</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.1727,0.1672,0.1661,0.2579,0.107,0.1742
Accuracy,0.8273,0.8328,0.8339,0.7421,0.893,0.8258


<span style="color:#666">**[</span><span style="color:#1776D2">2023-12-14 20:05:36</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.1742</span>, средняя точность: <span style="color:#1776D2">0.8258</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 364.619 сек. ---**</span>

True

<hr>

Для прогнозирования потребительских предпочтений в промышленных товарах необходимо знать коэффициенты корреляции, определяющие взаимосвязь между персональными качествами личности человека и предпочтениями в товарах или услугах.

В качестве примера предлагается использование коэффициентов корреляции между персональными качествами человека и характеристиками автомобилей, представленными в статье:

1) O'Connor P. J. et al. What Drives Consumer Automobile Choice? Investigating Personality Trait Predictors of Vehicle Preference Factors // Personality and Individual Differences. – 2022. – Vol. 184. – pp. 111220.

Пользователь может установить свои коэффициенты корреляции.

#### Прогнозирование потребительских предпочтений на характеристики атомобиля

In [13]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478675818?token=EjfLMqOeK8cfnOu&filename=auto_characteristics.csv'
df_correlation_coefficients = pd.read_csv(url)
df_correlation_coefficients = pd.DataFrame(
    df_correlation_coefficients.drop(['Style and performance', 'Safety and practicality'], axis = 1)
)
df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,Performance,Classic car features,Luxury additions,Fashion and attention,Recreation,Technology,Family friendly,Safe and reliable,Practical and easy to use,Economical/low cost,Basic features
ID,,,,,,,,,,,,
1,Openness,0.020000,-0.033333,-0.030000,-0.050000,0.033333,0.013333,-0.030000,0.136667,0.106667,0.093333,0.006667
2,Conscientiousness,0.013333,-0.193333,-0.063333,-0.096667,-0.096667,0.086667,-0.063333,0.280000,0.180000,0.130000,0.143333
3,Extraversion,0.133333,0.060000,0.106667,0.123333,0.126667,0.120000,0.090000,0.136667,0.043333,0.073333,0.050000
4,Agreeableness,-0.036667,-0.193333,-0.133333,-0.133333,-0.090000,0.046667,-0.016667,0.240000,0.160000,0.120000,0.083333
5,Non-Neuroticism,0.016667,-0.006667,-0.010000,-0.006667,-0.033333,0.046667,-0.023333,0.093333,0.046667,0.046667,-0.040000


In [14]:
_b5._priority_calculation(
    correlation_coefficients = df_correlation_coefficients,
    col_name_ocean = 'Trait',
    threshold = 0.55,
    number_priority = 3,
    number_importance_traits = 3,
    out = False
)

_b5._save_logs(df = _b5.df_files_priority_, name = 'auto_characteristics_priorities_mupta_en', out = True)

_b5.df_files_priority_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Priority 1,Priority 2,Priority 3,Trait importance 1,Trait importance 2,Trait importance 3
ID,,,,,,,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.564985,0.539052,0.440615,0.59251,0.488763,Practical and easy to use,Economical/low cost,Family friendly,Agreeableness,Openness,Non-Neuroticism
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.650774,0.663849,0.607308,0.643847,0.620627,Safe and reliable,Practical and easy to use,Economical/low cost,Conscientiousness,Agreeableness,Openness
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.435976,0.486683,0.313828,0.415446,0.396618,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.498542,0.511243,0.412592,0.468947,0.44399,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.394776,0.341608,0.327082,0.427304,0.354936,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.566107,0.543811,0.492766,0.587411,0.499433,Practical and easy to use,Economical/low cost,Family friendly,Agreeableness,Openness,Non-Neuroticism
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.506271,0.438215,0.430894,0.456177,0.44075,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.486463,0.521755,0.309894,0.432291,0.433601,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.417404,0.473339,0.320714,0.445086,0.414649,Classic car features,Fashion and attention,Luxury additions,Agreeableness,Conscientiousness,Openness


#### Прогнозирование потребительских предпочтений на характеристики мобильного девайса

В качестве примера предлагается использование коэффициентов корреляции между персональными качествами человека и характеристиками автомобилей, представленными в статье:

1) Peltonen E., Sharmila P., Asare K. O., Visuri A., Lagerspetz E., Ferreira D. (2020). When phones get personal: Predicting Big Five personality traits from application usage // Pervasive and Mobile Computing. – 2020. – Vol. 69. – 101269.

In [15]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478676690?token=7KcAxPqMpWiYQnx&filename=divice_characteristics.csv'
df_divice_characteristics = pd.read_csv(url)

df_divice_characteristics.index.name = 'ID'
df_divice_characteristics.index += 1
df_divice_characteristics.index = df_divice_characteristics.index.map(str)

df_divice_characteristics

,Trait,Communication,Game Action,Game Board,Game Casino,Game Educational,Game Simulation,Game Trivia,Entertainment,Finance,Health and Fitness,Media and Video,Music and Audio,News and Magazines,Personalisation,Travel and Local,Weather
ID,,,,,,,,,,,,,,,,,
1,Openness,0.118,0.056,0.079,0.342,0.027,0.104,0.026,0.000,0.006,0.002,0.000,0.000,0.001,0.004,0.002,0.004
2,Conscientiousness,0.119,0.043,0.107,0.448,0.039,0.012,0.119,0.000,0.005,0.001,0.000,0.002,0.002,0.001,0.001,0.003
3,Extraversion,0.246,0.182,0.211,0.311,0.102,0.165,0.223,0.001,0.003,0.000,0.001,0.001,0.001,0.004,0.009,0.003
4,Agreeableness,0.218,0.104,0.164,0.284,0.165,0.122,0.162,0.000,0.003,0.001,0.000,0.002,0.002,0.001,0.004,0.003
5,Non-Neuroticism,0.046,0.047,0.125,0.515,0.272,0.179,0.214,0.002,0.030,0.001,0.000,0.005,0.003,0.008,0.004,0.007


In [16]:
_b5._priority_calculation(
    correlation_coefficients = df_divice_characteristics,
    col_name_ocean = 'Trait',
    threshold = 0.55,
    number_priority = 3,
    number_importance_traits = 3,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_, name = 'divice_characteristics_priorities_mupta_en', out = True)

_b5.df_files_priority_

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Priority 1,Priority 2,Priority 3,Trait importance 1,Trait importance 2,Trait importance 3
ID,,,,,,,,,,,,
1,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.564985,0.539052,0.440615,0.59251,0.488763,Communication,Health and Fitness,Media and Video,Agreeableness,Openness,Non-Neuroticism
2,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.650774,0.663849,0.607308,0.643847,0.620627,Game Casino,Communication,Game Trivia,Non-Neuroticism,Extraversion,Conscientiousness
3,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.435976,0.486683,0.313828,0.415446,0.396618,Media and Video,Entertainment,Health and Fitness,Agreeableness,Conscientiousness,Extraversion
4,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.498542,0.511243,0.412592,0.468947,0.44399,Media and Video,Entertainment,Health and Fitness,Agreeableness,Conscientiousness,Extraversion
5,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.394776,0.341608,0.327082,0.427304,0.354936,Media and Video,Entertainment,Health and Fitness,Conscientiousness,Agreeableness,Extraversion
6,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.566107,0.543811,0.492766,0.587411,0.499433,Health and Fitness,Media and Video,News and Magazines,Agreeableness,Openness,Extraversion
7,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.506271,0.438215,0.430894,0.456177,0.44075,Media and Video,Entertainment,Health and Fitness,Conscientiousness,Agreeableness,Extraversion
8,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.486463,0.521755,0.309894,0.432291,0.433601,Media and Video,Entertainment,Health and Fitness,Agreeableness,Conscientiousness,Extraversion
9,D:\GitHub\OCEANAI\docs\source\user_guide\noteb...,0.417404,0.473339,0.320714,0.445086,0.414649,Media and Video,Entertainment,Health and Fitness,Agreeableness,Conscientiousness,Extraversion
